# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [1]:
# Install required package
# !pip install cognee

In [1]:
# Import required libraries
import os
import cognee
from cognee.shared.logging_utils import get_logger
from cognee.api.v1.search import SearchType

logger = get_logger()

# Set up OpenAI API key (required for Cognee's LLM functionality)
if "LLM_API_KEY" not in os.environ:
    os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key


2025-08-27T13:37:14.988241 [info     ] Deleted old log file: /Users/daulet/Desktop/dev/cognee-claude/logs/2025-08-26_12-45-20.log [cognee.shared.logging_utils]
/Users/daulet/Desktop/dev/cognee-claude/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

2025-08-27T13:37:16.002327 [info     ] Logging initialized            [cognee.shared.logging_utils] cognee_version=0.2.4-local database_path=/Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases graph_database_name= os_info='Darwin 24.5.0 (Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:43 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8132)' python_version=3.12.7 relational_config=cognee_db structlog_version=25.4.0 vector_config=lancedb

2025-08-27T13:37:16.003141 [info     ] Database storage: /Users/daulet/Desktop/dev/cognee-claude/

## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [2]:
async def run_pipeline(ontology_path=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(ontology_file_path=ontology_path)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [3]:
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(ontology_path=ontology_path)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")


--- Results WITH ontology ---




2025-08-27T13:37:26.484857 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]

2025-08-27T13:37:26.852005 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client
14:37:27 - LiteLLM:INFO: utils.py:3341 - 
LiteLLM completion() model= gpt-5-mini; provider = openai


LiteLLM completion() model= gpt-5-mini; provider = openai


User af34782a-1cf6-4632-8a3a-de25f57af415 has registered.



EmbeddingRateLimiter initialized: enabled=False, requests_limit=60, interval_seconds=60

2025-08-27T13:37:31.914752 [info     ] Pipeline run started: `49b3ca10-a420-5f3d-b7c1-88e5074b4697` [run_tasks_with_telemetry()]

2025-08-27T13:37:32.111670 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-08-27T13:37:32.294058 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-08-27T13:37:32.517354 [info     ] Registered loader: pypdf_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-08-27T13:37:32.518091 [info     ] Registered loader: text_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-08-27T13:37:32.519003 [info     ] Registered loader: image_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-08-27T13:37:32.519414 [info     ] Registered loader: audio_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-08-27T13:37:32.519853 [info     ] Registered loader: unstructured_loader [cognee.infrastructure.loaders.L

Q: What are common risk factors for Type 2 Diabetes?
A: ['Preparing brief answer using the provided context.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preparing a brief evidence-based answer from the provided context about measures that reduce risk of hypertension.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Acknowledged.']

Q: What diseases are associated with Obesity?
A: ['Acknowledged.']



In [4]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")


2025-08-27T13:42:17.046125 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]

2025-08-27T13:42:17.139414 [info     ] Database deleted successfully. [cognee.shared.logging_utils]



--- Results WITHOUT ontology ---




2025-08-27T13:42:17.270853 [info     ] Pipeline run started: `ab71824c-9eb0-5863-a3a2-0bfd328f0f8f` [run_tasks_with_telemetry()]

2025-08-27T13:42:17.424882 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]


User cbbb7ebc-556b-428f-9cf9-55bc1a277254 has registered.



2025-08-27T13:42:17.573292 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-08-27T13:42:17.752197 [info     ] Reading PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.pypdf_loader]

2025-08-27T13:42:17.879186 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]

2025-08-27T13:42:18.016115 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]

2025-08-27T13:42:18.164058 [info     ] Pipeline run completed: `ab71824c-9eb0-5863-a3a2-0bfd328f0f8f` [run_tasks_with_telemetry()]

2025-08-27T13:42:18.334120 [info     ] Pipeline run started: `ab71824c-9eb0-5863-a3a2-0bfd328f0f8f` [run_tasks_with_telemetry()]

2025-08-27T13:42:18.486127 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-08-27T13:42:18.628970 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-08-27T13:42:18.801360 [info     

Q: What are common risk factors for Type 2 Diabetes?
A: ['Using provided context to prepare a brief answer about common Type 2 diabetes risk factors.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preparing a brief, context-based answer on preventive measures that reduce risk of hypertension.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Common symptoms that can indicate possible cardiovascular disease:\n- Chest pain, pressure, squeezing or discomfort (angina)\n- Shortness of breath or difficulty breathing\n- Palpitations, an irregular or fast heartbeat (possible atrial fibrillation)\n- Lightheadedness, dizziness, or fainting (syncope)\n- Unexplained fatigue or reduced exercise tolerance\n- Swelling in the legs, ankles or abdomen (fluid retention/heart failure)\n- Sudden weakness, numbness, slurred speech, or vision changes (possible stroke)\n\nIf someone has chest pain, sudden severe shortness of breath, fainting, or stroke-like symptoms, see

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [5]:
import webbrowser
import os
from cognee.api.v1.visualize.visualize import visualize_graph
html = await visualize_graph()
home_dir = os.path.expanduser("~")
html_file = os.path.join(home_dir, "graph_visualization.html")
display(html_file)
webbrowser.open(f"file://{html_file}")


2025-08-27T13:46:35.623071 [info     ] Graph visualization saved as /Users/daulet/graph_visualization.html [cognee.shared.logging_utils]

2025-08-27T13:46:35.626149 [info     ] The HTML file has been stored on your home directory! Navigate there with cd ~ [cognee.shared.logging_utils]


'/Users/daulet/graph_visualization.html'

True

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key

In [ ]:
import os
os._exit(0)

: 